In [1]:
import os
import dotenv
import pandas as pd
from collections import Counter
import random

from src.constants import *
from src.openai_client import *
import numpy as np

from openai import OpenAI
from math import pi
import re
import matplotlib.pyplot as plt
from src.communication.messaging_service import SMSService
from src.communication.twilio_tool import NewTwilioConversationManager
from src.communication.reminder_manager import generate_personalised_reminder
from langchain_openai import ChatOpenAI
from src.prompts.prompts_templates import *
from src.data_manager.meal import get_recent_meals_string_for_user
from src.data_manager.nutritional_profiling import retrieve_nutritional_information, calculate_eating_consistency
from src.data_manager.reminder import get_previous_reminder_texts
from src.config.config import create_app
from src.utils.localization_utils import (
    get_localized_meal_type,
    get_localized_random_string,
    prompt_language_formatting
)


C:\Users\dryha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
def select_personalization_prompt(diet_info, last_used_prompt=None):
    """
    Selects an appropriate personalization prompt based on user's nutritional data.

    Args:
        user: User object with nutritional data
        last_used_prompt (str, optional): Identifier of the last used prompt to reduce repetition

    Returns:
        str: The selected personalization prompt
    """
    
    diet_info = str(diet_info).lower() 

    has_deficiency = "low" in diet_info
    has_excess = "high" in diet_info
    
    # Prompt pool with tags
    prompt_pool = [
        {"prompt": None, "tag": "skip", "weight": 0.5}, 
        {"prompt": NUTRITIONAL_DEFICIENCY_PROMPT, "tag": "deficiency", "weight": 1.5 if has_deficiency else 0.2},
        {"prompt": NUTRITIONAL_EXCESS_PROMPT, "tag": "excess", "weight": 1.5 if has_excess else 0.2},
        {"prompt": FUN_FACT_ENCOURAGEMENT_PROMPT, "tag": "fun_fact", "weight": 1 if not has_deficiency and not has_excess else 0.5},
    ]

    # Optional: reduce repetition of the last-used prompt
    if last_used_prompt:
        for entry in prompt_pool:
            if entry["prompt"] == last_used_prompt:
                entry["weight"] *= 0.3  
    # Normalize and draw
    prompts, weights = zip(*[(entry["prompt"], entry["weight"]) for entry in prompt_pool])
    selected_prompt = random.choices(prompts, weights=weights, k=1)[0]

    return selected_prompt

def add_cot(prompt):
    if prompt == FUN_FACT_ENCOURAGEMENT_PROMPT :
        return FUN_FACT_ENCOURAGEMENT_COT
    elif prompt == NUTRITIONAL_DEFICIENCY_PROMPT:
        return NUTRITIONAL_DEFICIENCY_COT
    elif prompt == NUTRITIONAL_EXCESS_PROMPT :
        return NUTRITIONAL_EXCESS_COT
    else :
        return ""

In [4]:
dotenv.load_dotenv()
openai.api_key = os.environ.get('OPENAI_API_KEY')
models = [GPT_4_O,GPT_4_TURBO,GPT_4_1_MINI]
user_data = {
    "meal_type": "lunch",
    "age": "49",
    "gender": "M",
    "diet_goal": "I try to get enough protein (about 160g) and not more than 70g fat. I try to eat as healthy as possible according to scientific evidence.",
    "recent_meals": list(reversed([
                "MOST RECENT MEAL: Food Item: Riegel high protein coconut almond bar, Skyr, Naturaplan : Drink : Lait partiellement écrémé, Kellogg's Special K - Protein Nuts, clusters and seeds Time: 2025-05-16 09:02 nutrients: - Energy_kcal: 606.55 - Fat: 15.55 - Fatty_acids_saturated: 6.01 - Carbohydrates: 68.95 - Sugar: 23.32 - Salt: 1.0 - Fiber: 7.53 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 66.15 - Water: 0.0 - Protein: 46.25 - Cholesterol: 0.0 PREVIOUS MEAL #1: Food Item: Tomato sauce, Dried pasta, raw, Isey Skyr Protein Drink Strawberry-Banana, Dried meat (Hobel-, Bündner-, Walliser-, Tessiner- dried meat) Time: 2025-05-15 20:23 nutrients: - Energy_kcal: 1056.5 - Fat: 9.31 - Fatty_acids_saturated: 2.76 - Carbohydrates: 159.68 - Sugar: 25.52 - Salt: 5.56 - Fiber: 7.9 - Calcium: 64.0 - Iron: 8.2 - Zinc: 9.19 - Phosphorus: 676.5 - Fatty_acids_polyunsaturated: 2.31 - Fatty_acids_monounsaturated: 3.11 - Alcohol: 0.0 - Starch: 130.5 - Sodium: 2097.4 - Water: 166.29 - Protein: 86.13 - Cholesterol: 79.8 PREVIOUS MEAL #2: Food Item: Drinking water (average Switzerland), Red onion, raw, Tomato, raw, Lettuce, raw, Tzatziki, from yogurt, prepacked, Poultry on skewer, with vegetables (onion, sweet pepper…), cooked, Pita bread, French fries (deep fried in HO sunflower oil), unsalted, Apple, fresh Time: 2025-05-15 11:59 nutrients: - Energy_kcal: 686.86 - Fat: 23.34 - Fatty_acids_saturated: 2.81 - Carbohydrates: 89.35 - Sugar: 21.44 - Salt: 1.77 - Fiber: 7.01 - Calcium: 124.23 - Iron: 2.04 - Zinc: 1.4 - Phosphorus: 329.04 - Fatty_acids_polyunsaturated: 2.78 - Fatty_acids_monounsaturated: 16.29 - Alcohol: 0.0 - Starch: 64.54 - Sodium: 720.91 - Water: 503.99 - Protein: 25.73 - Cholesterol: 49.65 PREVIOUS MEAL #3: Food Item: Naturaplan : Drink : Lait partiellement écrémé, Kellogg's Special K - Protein Nuts, clusters and seeds, Skyr Time: 2025-05-15 07:53 nutrients: - Energy_kcal: 473.9 - Fat: 10.3 - Fatty_acids_saturated: 3.42 - Carbohydrates: 61.6 - Sugar: 23.0 - Salt: 0.83 - Fiber: 5.81 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 30.5 - Cholesterol: 0.0 PREVIOUS MEAL #4: Food Item: Couscous w chicken, Dried pasta, cooked, unsalted Time: 2025-05-14 19:16 nutrients: - Energy_kcal: 385.2 - Fat: 7.59 - Fatty_acids_saturated: 2.24 - Carbohydrates: 58.08 - Sugar: 5.89 - Salt: 1.42 - Fiber: 4.93 - Calcium: 53.52 - Iron: 2.07 - Zinc: 0.56 - Phosphorus: 195.0 - Fatty_acids_polyunsaturated: 1.79 - Fatty_acids_monounsaturated: 3.01 - Alcohol: 0.0 - Starch: 49.08 - Sodium: 552.6 - Water: 202.38 - Protein: 18.29 - Cholesterol: 49.4 PREVIOUS MEAL #5: Food Item: Oh! - High protein drink, banane Time: 2025-05-14 14:51 nutrients: - Energy_kcal: 122.5 - Fat: 1.25 - Fatty_acids_saturated: 1.25 - Carbohydrates: 12.5 - Sugar: 12.5 - Salt: 0.25 - Fiber: 1.25 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 17.5 - Cholesterol: 0.0 PREVIOUS MEAL #6: Food Item: Mix of unsalted grains/nuts and dried fruit, Cranberry, fresh, Mueslimix, crunchy, with nuts and seeds, sweetened , Yogurt natural Time: 2025-05-14 12:19 nutrients: - Energy_kcal: 231.31 - Fat: 10.5 - Fatty_acids_saturated: 3.85 - Carbohydrates: 24.34 - Sugar: 10.77 - Salt: 0.22 - Fiber: 3.01 - Calcium: 187.32 - Iron: 1.08 - Zinc: 1.26 - Phosphorus: 222.8 - Fatty_acids_polyunsaturated: 2.29 - Fatty_acids_monounsaturated: 3.55 - Alcohol: 0.0 - Starch: 12.98 - Sodium: 103.81 - Water: 113.83 - Protein: 8.5 - Cholesterol: 13.2 PREVIOUS MEAL #7: Food Item: Cereal bar, with chocolate coating Time: 2025-05-14 12:00 nutrients: - Energy_kcal: 229.5 - Fat: 10.1 - Fatty_acids_saturated: 5.45 - Carbohydrates: 29.0 - Sugar: 18.55 - Salt: 0.2 - Fiber: 2.85 - Calcium: 55.0 - Iron: 1.1 - Zinc: 0.75 - Phosphorus: 105.0 - Fatty_acids_polyunsaturated: 1.0 - Fatty_acids_monounsaturated: 3.3 - Alcohol: 0.0 - Starch: 9.95 - Sodium: 75.0 - Water: 3.2 - Protein: 4.25 - Cholesterol: 2.0 PREVIOUS MEAL #8: Food Item: French omelette, prepared Time: 2025-05-14 08:55 nutrients: - Energy_kcal: 181.2 - Fat: 13.32 - Fatty_acids_saturated: 4.56 - Carbohydrates: 1.68 - Sugar: 1.68 - Salt: 0.96 - Fiber: 0.0 - Calcium: 84.0 - Iron: 1.8 - Zinc: 1.32 - Phosphorus: 228.0 - Fatty_acids_polyunsaturated: 1.8 - Fatty_acids_monounsaturated: 4.56 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 408.0 - Water: 91.32 - Protein: 13.68 - Cholesterol: 372.0 PREVIOUS MEAL #9: Food Item: Chives, fresh, Butter of choice, Cream cheese, min. 60% fidm, Pineapple, fresh, Wholewheat bread, Salmon, smoked Time: 2025-05-14 08:07 nutrients: - Energy_kcal: 297.75 - Fat: 16.9 - Fatty_acids_saturated: 8.62 - Carbohydrates: 20.18 - Sugar: 4.37 - Salt: 2.4 - Fiber: 3.2 - Calcium: 36.06 - Iron: 1.35 - Zinc: 1.15 - Phosphorus: 236.34 - Fatty_acids_polyunsaturated: 2.05 - Fatty_acids_monounsaturated: 4.17 - Alcohol: 0.0 - Starch: 15.04 - Sodium: 971.03 - Water: 83.79 - Protein: 14.66 - Cholesterol: 58.5 PREVIOUS MEAL #10: Food Item: Beer, lager, Meringue, biscuit, Cake (average), Mushroom (average), raw, French fries (deep fried in HO sunflower oil), unsalted, Leafy salad (average), raw , Duck, meat, roasted/baked, Brioche, Strawberry, fresh, Watermelon, fresh Time: 2025-05-13 19:06 nutrients: - Energy_kcal: 1134.95 - Fat: 37.6 - Fatty_acids_saturated: 10.53 - Carbohydrates: 110.52 - Sugar: 56.59 - Salt: 1.28 - Fiber: 6.75 - Calcium: 119.76 - Iron: 6.4 - Zinc: 3.26 - Phosphorus: 561.2 - Fatty_acids_polyunsaturated: 3.25 - Fatty_acids_monounsaturated: 16.07 - Alcohol: 26.48 - Starch: 41.29 - Sodium: 541.4 - Water: 918.54 - Protein: 39.72 - Cholesterol: 207.82"
                "MOST RECENT MEAL: Food Item: Drinking water (average Switzerland), Red onion, raw, Tomato, raw, Lettuce, raw, Tzatziki, from yogurt, prepacked, Poultry on skewer, with vegetables (onion, sweet pepper…), cooked, Pita bread, French fries (deep fried in HO sunflower oil), unsalted, Apple, fresh Time: 2025-05-15 11:59 nutrients: - Energy_kcal: 686.86 - Fat: 23.34 - Fatty_acids_saturated: 2.81 - Carbohydrates: 89.35 - Sugar: 21.44 - Salt: 1.77 - Fiber: 7.01 - Calcium: 124.23 - Iron: 2.04 - Zinc: 1.4 - Phosphorus: 329.04 - Fatty_acids_polyunsaturated: 2.78 - Fatty_acids_monounsaturated: 16.29 - Alcohol: 0.0 - Starch: 64.54 - Sodium: 720.91 - Water: 503.99 - Protein: 25.73 - Cholesterol: 49.65 PREVIOUS MEAL #1: Food Item: Naturaplan : Drink : Lait partiellement écrémé, Skyr, Kellogg's Special K - Protein Nuts, clusters and seeds Time: 2025-05-15 07:53 nutrients: - Energy_kcal: 473.9 - Fat: 10.3 - Fatty_acids_saturated: 3.42 - Carbohydrates: 61.6 - Sugar: 23.0 - Salt: 0.83 - Fiber: 5.81 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 30.5 - Cholesterol: 0.0 PREVIOUS MEAL #2: Food Item: Couscous w chicken, Dried pasta, cooked, unsalted Time: 2025-05-14 19:16 nutrients: - Energy_kcal: 385.2 - Fat: 7.59 - Fatty_acids_saturated: 2.24 - Carbohydrates: 58.08 - Sugar: 5.89 - Salt: 1.42 - Fiber: 4.93 - Calcium: 53.52 - Iron: 2.07 - Zinc: 0.56 - Phosphorus: 195.0 - Fatty_acids_polyunsaturated: 1.79 - Fatty_acids_monounsaturated: 3.01 - Alcohol: 0.0 - Starch: 49.08 - Sodium: 552.6 - Water: 202.38 - Protein: 18.29 - Cholesterol: 49.4 PREVIOUS MEAL #3: Food Item: Oh! - High protein drink, banane Time: 2025-05-14 14:51 nutrients: - Energy_kcal: 122.5 - Fat: 1.25 - Fatty_acids_saturated: 1.25 - Carbohydrates: 12.5 - Sugar: 12.5 - Salt: 0.25 - Fiber: 1.25 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 17.5 - Cholesterol: 0.0 PREVIOUS MEAL #4: Food Item: Mix of unsalted grains/nuts and dried fruit, Cranberry, fresh, Mueslimix, crunchy, with nuts and seeds, sweetened , Yogurt natural Time: 2025-05-14 12:19 nutrients: - Energy_kcal: 231.31 - Fat: 10.5 - Fatty_acids_saturated: 3.85 - Carbohydrates: 24.34 - Sugar: 10.77 - Salt: 0.22 - Fiber: 3.01 - Calcium: 187.32 - Iron: 1.08 - Zinc: 1.26 - Phosphorus: 222.8 - Fatty_acids_polyunsaturated: 2.29 - Fatty_acids_monounsaturated: 3.55 - Alcohol: 0.0 - Starch: 12.98 - Sodium: 103.81 - Water: 113.83 - Protein: 8.5 - Cholesterol: 13.2 PREVIOUS MEAL #5: Food Item: Cereal bar, with chocolate coating Time: 2025-05-14 12:00 nutrients: - Energy_kcal: 229.5 - Fat: 10.1 - Fatty_acids_saturated: 5.45 - Carbohydrates: 29.0 - Sugar: 18.55 - Salt: 0.2 - Fiber: 2.85 - Calcium: 55.0 - Iron: 1.1 - Zinc: 0.75 - Phosphorus: 105.0 - Fatty_acids_polyunsaturated: 1.0 - Fatty_acids_monounsaturated: 3.3 - Alcohol: 0.0 - Starch: 9.95 - Sodium: 75.0 - Water: 3.2 - Protein: 4.25 - Cholesterol: 2.0 PREVIOUS MEAL #6: Food Item: French omelette, prepared Time: 2025-05-14 08:55 nutrients: - Energy_kcal: 181.2 - Fat: 13.32 - Fatty_acids_saturated: 4.56 - Carbohydrates: 1.68 - Sugar: 1.68 - Salt: 0.96 - Fiber: 0.0 - Calcium: 84.0 - Iron: 1.8 - Zinc: 1.32 - Phosphorus: 228.0 - Fatty_acids_polyunsaturated: 1.8 - Fatty_acids_monounsaturated: 4.56 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 408.0 - Water: 91.32 - Protein: 13.68 - Cholesterol: 372.0 PREVIOUS MEAL #7: Food Item: Chives, fresh, Butter of choice, Cream cheese, min. 60% fidm, Pineapple, fresh, Wholewheat bread, Salmon, smoked Time: 2025-05-14 08:07 nutrients: - Energy_kcal: 297.75 - Fat: 16.9 - Fatty_acids_saturated: 8.62 - Carbohydrates: 20.18 - Sugar: 4.37 - Salt: 2.4 - Fiber: 3.2 - Calcium: 36.06 - Iron: 1.35 - Zinc: 1.15 - Phosphorus: 236.34 - Fatty_acids_polyunsaturated: 2.05 - Fatty_acids_monounsaturated: 4.17 - Alcohol: 0.0 - Starch: 15.04 - Sodium: 971.03 - Water: 83.79 - Protein: 14.66 - Cholesterol: 58.5 PREVIOUS MEAL #8: Food Item: Beer, lager, Meringue, biscuit, Cake (average), Mushroom (average), raw, French fries (deep fried in HO sunflower oil), unsalted, Leafy salad (average), raw , Duck, meat, roasted/baked, Brioche, Strawberry, fresh, Watermelon, fresh Time: 2025-05-13 19:06 nutrients: - Energy_kcal: 1134.95 - Fat: 37.6 - Fatty_acids_saturated: 10.53 - Carbohydrates: 110.52 - Sugar: 56.59 - Salt: 1.28 - Fiber: 6.75 - Calcium: 119.76 - Iron: 6.4 - Zinc: 3.26 - Phosphorus: 561.2 - Fatty_acids_polyunsaturated: 3.25 - Fatty_acids_monounsaturated: 16.07 - Alcohol: 26.48 - Starch: 41.29 - Sodium: 541.4 - Water: 918.54 - Protein: 39.72 - Cholesterol: 207.82 PREVIOUS MEAL #9: Food Item: Salmon, smoked, Sandwich loaf, wholemeal, Braided white bread made with butter Time: 2025-05-13 12:30 nutrients: - Energy_kcal: 686.45 - Fat: 21.59 - Fatty_acids_saturated: 10.78 - Carbohydrates: 94.77 - Sugar: 7.76 - Salt: 3.98 - Fiber: 6.89 - Calcium: 160.6 - Iron: 1.71 - Zinc: 1.66 - Phosphorus: 308.5 - Fatty_acids_polyunsaturated: 2.49 - Fatty_acids_monounsaturated: 5.31 - Alcohol: 0.12 - Starch: 84.42 - Sodium: 1582.6 - Water: 85.61 - Protein: 24.41 - Cholesterol: 74.0 PREVIOUS MEAL #10: Food Item: Chiefs - Protein bar, crispy cookie Time: 2025-05-13 10:49 nutrients: - Energy_kcal: 210.65 - Fat: 8.25 - Fatty_acids_saturated: 5.0 - Carbohydrates: 20.35 - Sugar: 1.59 - Salt: 0.45 - Fiber: 0.94 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 20.35 - Cholesterol: 0.0",
                "MOST RECENT MEAL: Food Item: Couscous w chicken, Dried pasta, cooked, unsalted Time: 2025-05-14 19:16 nutrients: - Energy_kcal: 385.2 - Fat: 7.59 - Fatty_acids_saturated: 2.24 - Carbohydrates: 58.08 - Sugar: 5.89 - Salt: 1.42 - Fiber: 4.93 - Calcium: 53.52 - Iron: 2.07 - Zinc: 0.56 - Phosphorus: 195.0 - Fatty_acids_polyunsaturated: 1.79 - Fatty_acids_monounsaturated: 3.01 - Alcohol: 0.0 - Starch: 49.08 - Sodium: 552.6 - Water: 202.38 - Protein: 18.29 - Cholesterol: 49.4 PREVIOUS MEAL #1: Food Item: Oh! - High protein drink, banane Time: 2025-05-14 14:51 nutrients: - Energy_kcal: 122.5 - Fat: 1.25 - Fatty_acids_saturated: 1.25 - Carbohydrates: 12.5 - Sugar: 12.5 - Salt: 0.25 - Fiber: 1.25 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 17.5 - Cholesterol: 0.0 PREVIOUS MEAL #2: Food Item: Mix of unsalted grains/nuts and dried fruit, Cranberry, fresh, Mueslimix, crunchy, with nuts and seeds, sweetened , Yogurt natural Time: 2025-05-14 12:19 nutrients: - Energy_kcal: 231.31 - Fat: 10.5 - Fatty_acids_saturated: 3.85 - Carbohydrates: 24.34 - Sugar: 10.77 - Salt: 0.22 - Fiber: 3.01 - Calcium: 187.32 - Iron: 1.08 - Zinc: 1.26 - Phosphorus: 222.8 - Fatty_acids_polyunsaturated: 2.29 - Fatty_acids_monounsaturated: 3.55 - Alcohol: 0.0 - Starch: 12.98 - Sodium: 103.81 - Water: 113.83 - Protein: 8.5 - Cholesterol: 13.2 PREVIOUS MEAL #3: Food Item: Cereal bar, with chocolate coating Time: 2025-05-14 12:00 nutrients: - Energy_kcal: 229.5 - Fat: 10.1 - Fatty_acids_saturated: 5.45 - Carbohydrates: 29.0 - Sugar: 18.55 - Salt: 0.2 - Fiber: 2.85 - Calcium: 55.0 - Iron: 1.1 - Zinc: 0.75 - Phosphorus: 105.0 - Fatty_acids_polyunsaturated: 1.0 - Fatty_acids_monounsaturated: 3.3 - Alcohol: 0.0 - Starch: 9.95 - Sodium: 75.0 - Water: 3.2 - Protein: 4.25 - Cholesterol: 2.0 PREVIOUS MEAL #4: Food Item: Chives, fresh, Butter of choice, Cream cheese, min. 60% fidm, Pineapple, fresh, Wholewheat bread, Salmon, smoked Time: 2025-05-14 08:07 nutrients: - Energy_kcal: 297.75 - Fat: 16.9 - Fatty_acids_saturated: 8.62 - Carbohydrates: 20.18 - Sugar: 4.37 - Salt: 2.4 - Fiber: 3.2 - Calcium: 36.06 - Iron: 1.35 - Zinc: 1.15 - Phosphorus: 236.34 - Fatty_acids_polyunsaturated: 2.05 - Fatty_acids_monounsaturated: 4.17 - Alcohol: 0.0 - Starch: 15.04 - Sodium: 971.03 - Water: 83.79 - Protein: 14.66 - Cholesterol: 58.5 PREVIOUS MEAL #5: Food Item: Beer, lager, Meringue, biscuit, Cake (average), Mushroom (average), raw, French fries (deep fried in HO sunflower oil), unsalted, Leafy salad (average), raw , Duck, meat, roasted/baked, Brioche, Strawberry, fresh, Watermelon, fresh Time: 2025-05-13 19:06 nutrients: - Energy_kcal: 1134.95 - Fat: 37.6 - Fatty_acids_saturated: 10.53 - Carbohydrates: 110.52 - Sugar: 56.59 - Salt: 1.28 - Fiber: 6.75 - Calcium: 119.76 - Iron: 6.4 - Zinc: 3.26 - Phosphorus: 561.2 - Fatty_acids_polyunsaturated: 3.25 - Fatty_acids_monounsaturated: 16.07 - Alcohol: 26.48 - Starch: 41.29 - Sodium: 541.4 - Water: 918.54 - Protein: 39.72 - Cholesterol: 207.82 PREVIOUS MEAL #6: Food Item: Salmon, smoked, Sandwich loaf, wholemeal, Braided white bread made with butter Time: 2025-05-13 12:30 nutrients: - Energy_kcal: 686.45 - Fat: 21.59 - Fatty_acids_saturated: 10.78 - Carbohydrates: 94.77 - Sugar: 7.76 - Salt: 3.98 - Fiber: 6.89 - Calcium: 160.6 - Iron: 1.71 - Zinc: 1.66 - Phosphorus: 308.5 - Fatty_acids_polyunsaturated: 2.49 - Fatty_acids_monounsaturated: 5.31 - Alcohol: 0.12 - Starch: 84.42 - Sodium: 1582.6 - Water: 85.61 - Protein: 24.41 - Cholesterol: 74.0 PREVIOUS MEAL #7: Food Item: Chiefs - Protein bar, crispy cookie Time: 2025-05-13 10:49 nutrients: - Energy_kcal: 210.65 - Fat: 8.25 - Fatty_acids_saturated: 5.0 - Carbohydrates: 20.35 - Sugar: 1.59 - Salt: 0.45 - Fiber: 0.94 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 20.35 - Cholesterol: 0.0 PREVIOUS MEAL #8: Food Item: Mineral still water (Evian), bottled, lightly mineralized, Pineapple, pulp, raw Time: 2025-05-13 09:52 nutrients: - Energy_kcal: 48.96 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 10.53 - Sugar: 9.45 - Salt: 0.0 - Fiber: 1.08 - Calcium: 13.2 - Iron: 0.15 - Zinc: 0.07 - Phosphorus: 7.29 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.49 - Water: 151.95 - Protein: 0.0 - Cholesterol: 0.0 PREVIOUS MEAL #9: Food Item: Skyr, Naturaplan : Drink : Lait partiellement écrémé, Kellogg's Special K - Protein Nuts, clusters and seeds Time: 2025-05-13 07:01 nutrients: - Energy_kcal: 473.9 - Fat: 10.3 - Fatty_acids_saturated: 3.42 - Carbohydrates: 61.6 - Sugar: 23.0 - Salt: 0.83 - Fiber: 5.81 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 30.5 - Cholesterol: 0.0 PREVIOUS MEAL #10: Food Item: Skyr, Meal replacement low calorie, in powder, reconstituted with skimmed milk, milkshake type, Beer, lager Time: 2025-05-12 22:50 nutrients: - Energy_kcal: 472.5 - Fat: 6.84 - Fatty_acids_saturated: 0.84 - Carbohydrates: 46.2 - Sugar: 41.4 - Salt: 0.69 - Fiber: 1.89 - Calcium: 526.5 - Iron: 5.73 - Zinc: 9.39 - Phosphorus: 471.0 - Fatty_acids_polyunsaturated: 3.69 - Fatty_acids_monounsaturated: 1.98 - Alcohol: 12.0 - Starch: 0.6 - Sodium: 237.6 - Water: 515.7 - Protein: 35.07 - Cholesterol: 30.0",
                "MOST RECENT MEAL: Food Item: Oh! - High protein drink, banane Time: 2025-05-14 14:51 nutrients: - Energy_kcal: 122.5 - Fat: 1.25 - Fatty_acids_saturated: 1.25 - Carbohydrates: 12.5 - Sugar: 12.5 - Salt: 0.25 - Fiber: 1.25 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 17.5 - Cholesterol: 0.0 PREVIOUS MEAL #1: Food Item: Mix of unsalted grains/nuts and dried fruit, Cranberry, fresh, Mueslimix, crunchy, with nuts and seeds, sweetened , Yogurt natural Time: 2025-05-14 12:19 nutrients: - Energy_kcal: 231.31 - Fat: 10.5 - Fatty_acids_saturated: 3.85 - Carbohydrates: 24.34 - Sugar: 10.77 - Salt: 0.22 - Fiber: 3.01 - Calcium: 187.32 - Iron: 1.08 - Zinc: 1.26 - Phosphorus: 222.8 - Fatty_acids_polyunsaturated: 2.29 - Fatty_acids_monounsaturated: 3.55 - Alcohol: 0.0 - Starch: 12.98 - Sodium: 103.81 - Water: 113.83 - Protein: 8.5 - Cholesterol: 13.2 PREVIOUS MEAL #2: Food Item: Cereal bar, with chocolate coating Time: 2025-05-14 12:00 nutrients: - Energy_kcal: 229.5 - Fat: 10.1 - Fatty_acids_saturated: 5.45 - Carbohydrates: 29.0 - Sugar: 18.55 - Salt: 0.2 - Fiber: 2.85 - Calcium: 55.0 - Iron: 1.1 - Zinc: 0.75 - Phosphorus: 105.0 - Fatty_acids_polyunsaturated: 1.0 - Fatty_acids_monounsaturated: 3.3 - Alcohol: 0.0 - Starch: 9.95 - Sodium: 75.0 - Water: 3.2 - Protein: 4.25 - Cholesterol: 2.0 PREVIOUS MEAL #3: Food Item: Chives, fresh, Butter of choice, Cream cheese, min. 60% fidm, Pineapple, fresh, Wholewheat bread, Salmon, smoked Time: 2025-05-14 08:07 nutrients: - Energy_kcal: 297.75 - Fat: 16.9 - Fatty_acids_saturated: 8.62 - Carbohydrates: 20.18 - Sugar: 4.37 - Salt: 2.4 - Fiber: 3.2 - Calcium: 36.06 - Iron: 1.35 - Zinc: 1.15 - Phosphorus: 236.34 - Fatty_acids_polyunsaturated: 2.05 - Fatty_acids_monounsaturated: 4.17 - Alcohol: 0.0 - Starch: 15.04 - Sodium: 971.03 - Water: 83.79 - Protein: 14.66 - Cholesterol: 58.5 PREVIOUS MEAL #4: Food Item: Beer, lager, Meringue, biscuit, Cake (average), Mushroom (average), raw, French fries (deep fried in HO sunflower oil), unsalted, Leafy salad (average), raw , Duck, meat, roasted/baked, Brioche, Strawberry, fresh, Watermelon, fresh Time: 2025-05-13 19:06 nutrients: - Energy_kcal: 1134.95 - Fat: 37.6 - Fatty_acids_saturated: 10.53 - Carbohydrates: 110.52 - Sugar: 56.59 - Salt: 1.28 - Fiber: 6.75 - Calcium: 119.76 - Iron: 6.4 - Zinc: 3.26 - Phosphorus: 561.2 - Fatty_acids_polyunsaturated: 3.25 - Fatty_acids_monounsaturated: 16.07 - Alcohol: 26.48 - Starch: 41.29 - Sodium: 541.4 - Water: 918.54 - Protein: 39.72 - Cholesterol: 207.82 PREVIOUS MEAL #5: Food Item: Salmon, smoked, Sandwich loaf, wholemeal, Braided white bread made with butter Time: 2025-05-13 12:30 nutrients: - Energy_kcal: 686.45 - Fat: 21.59 - Fatty_acids_saturated: 10.78 - Carbohydrates: 94.77 - Sugar: 7.76 - Salt: 3.98 - Fiber: 6.89 - Calcium: 160.6 - Iron: 1.71 - Zinc: 1.66 - Phosphorus: 308.5 - Fatty_acids_polyunsaturated: 2.49 - Fatty_acids_monounsaturated: 5.31 - Alcohol: 0.12 - Starch: 84.42 - Sodium: 1582.6 - Water: 85.61 - Protein: 24.41 - Cholesterol: 74.0 PREVIOUS MEAL #6: Food Item: Chiefs - Protein bar, crispy cookie Time: 2025-05-13 10:49 nutrients: - Energy_kcal: 210.65 - Fat: 8.25 - Fatty_acids_saturated: 5.0 - Carbohydrates: 20.35 - Sugar: 1.59 - Salt: 0.45 - Fiber: 0.94 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 20.35 - Cholesterol: 0.0 PREVIOUS MEAL #7: Food Item: Mineral still water (Evian), bottled, lightly mineralized, Pineapple, pulp, raw Time: 2025-05-13 09:52 nutrients: - Energy_kcal: 48.96 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 10.53 - Sugar: 9.45 - Salt: 0.0 - Fiber: 1.08 - Calcium: 13.2 - Iron: 0.15 - Zinc: 0.07 - Phosphorus: 7.29 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.49 - Water: 151.95 - Protein: 0.0 - Cholesterol: 0.0 PREVIOUS MEAL #8: Food Item: Naturaplan : Drink : Lait partiellement écrémé, Skyr, Kellogg's Special K - Protein Nuts, clusters and seeds Time: 2025-05-13 07:01 nutrients: - Energy_kcal: 473.9 - Fat: 10.3 - Fatty_acids_saturated: 3.42 - Carbohydrates: 61.6 - Sugar: 23.0 - Salt: 0.83 - Fiber: 5.81 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 30.5 - Cholesterol: 0.0 PREVIOUS MEAL #9: Food Item: Skyr, Meal replacement low calorie, in powder, reconstituted with skimmed milk, milkshake type, Beer, lager Time: 2025-05-12 22:50 nutrients: - Energy_kcal: 472.5 - Fat: 6.84 - Fatty_acids_saturated: 0.84 - Carbohydrates: 46.2 - Sugar: 41.4 - Salt: 0.69 - Fiber: 1.89 - Calcium: 526.5 - Iron: 5.73 - Zinc: 9.39 - Phosphorus: 471.0 - Fatty_acids_polyunsaturated: 3.69 - Fatty_acids_monounsaturated: 1.98 - Alcohol: 12.0 - Starch: 0.6 - Sodium: 237.6 - Water: 515.7 - Protein: 35.07 - Cholesterol: 30.0 PREVIOUS MEAL #10: Food Item: Broccoli, raw, Rice-maize based pasta, gluten-free, boiled, Fish, breaded, fried Time: 2025-05-12 19:12 nutrients: - Energy_kcal: 223.9 - Fat: 1.42 - Fatty_acids_saturated: 0.47 - Carbohydrates: 23.56 - Sugar: 0.95 - Salt: 0.08 - Fiber: 2.38 - Calcium: 32.04 - Iron: 0.85 - Zinc: 0.84 - Phosphorus: 240.2 - Fatty_acids_polyunsaturated: 0.38 - Fatty_acids_monounsaturated: 0.45 - Alcohol: 0.0 - Starch: 22.02 - Sodium: 36.4 - Water: 134.18 - Protein: 27.77 - Cholesterol: 63.2",
                "MOST RECENT MEAL: Food Item: Chives, fresh, Butter of choice, Cream cheese, min. 60% fidm, Pineapple, fresh, Wholewheat bread, Salmon, smoked Time: 2025-05-14 08:07 nutrients: - Energy_kcal: 297.75 - Fat: 16.9 - Fatty_acids_saturated: 8.62 - Carbohydrates: 20.18 - Sugar: 4.37 - Salt: 2.4 - Fiber: 3.2 - Calcium: 36.06 - Iron: 1.35 - Zinc: 1.15 - Phosphorus: 236.34 - Fatty_acids_polyunsaturated: 2.05 - Fatty_acids_monounsaturated: 4.17 - Alcohol: 0.0 - Starch: 15.04 - Sodium: 971.03 - Water: 83.79 - Protein: 14.66 - Cholesterol: 58.5 PREVIOUS MEAL #1: Food Item: Salmon, smoked, Sandwich loaf, wholemeal, Braided white bread made with butter Time: 2025-05-13 12:30 nutrients: - Energy_kcal: 686.45 - Fat: 21.59 - Fatty_acids_saturated: 10.78 - Carbohydrates: 94.77 - Sugar: 7.76 - Salt: 3.98 - Fiber: 6.89 - Calcium: 160.6 - Iron: 1.71 - Zinc: 1.66 - Phosphorus: 308.5 - Fatty_acids_polyunsaturated: 2.49 - Fatty_acids_monounsaturated: 5.31 - Alcohol: 0.12 - Starch: 84.42 - Sodium: 1582.6 - Water: 85.61 - Protein: 24.41 - Cholesterol: 74.0 PREVIOUS MEAL #2: Food Item: Chiefs - Protein bar, crispy cookie Time: 2025-05-13 10:49 nutrients: - Energy_kcal: 210.65 - Fat: 8.25 - Fatty_acids_saturated: 5.0 - Carbohydrates: 20.35 - Sugar: 1.59 - Salt: 0.45 - Fiber: 0.94 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 20.35 - Cholesterol: 0.0 PREVIOUS MEAL #3: Food Item: Mineral still water (Evian), bottled, lightly mineralized, Pineapple, pulp, raw Time: 2025-05-13 09:52 nutrients: - Energy_kcal: 48.96 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 10.53 - Sugar: 9.45 - Salt: 0.0 - Fiber: 1.08 - Calcium: 13.2 - Iron: 0.15 - Zinc: 0.07 - Phosphorus: 7.29 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.49 - Water: 151.95 - Protein: 0.0 - Cholesterol: 0.0 PREVIOUS MEAL #4: Food Item: Skyr, Meal replacement low calorie, in powder, reconstituted with skimmed milk, milkshake type, Beer, lager Time: 2025-05-12 22:50 nutrients: - Energy_kcal: 472.5 - Fat: 6.84 - Fatty_acids_saturated: 0.84 - Carbohydrates: 46.2 - Sugar: 41.4 - Salt: 0.69 - Fiber: 1.89 - Calcium: 526.5 - Iron: 5.73 - Zinc: 9.39 - Phosphorus: 471.0 - Fatty_acids_polyunsaturated: 3.69 - Fatty_acids_monounsaturated: 1.98 - Alcohol: 12.0 - Starch: 0.6 - Sodium: 237.6 - Water: 515.7 - Protein: 35.07 - Cholesterol: 30.0 PREVIOUS MEAL #5: Food Item: Broccoli, raw, Rice-maize based pasta, gluten-free, boiled, Fish, breaded, fried Time: 2025-05-12 19:12 nutrients: - Energy_kcal: 223.9 - Fat: 1.42 - Fatty_acids_saturated: 0.47 - Carbohydrates: 23.56 - Sugar: 0.95 - Salt: 0.08 - Fiber: 2.38 - Calcium: 32.04 - Iron: 0.85 - Zinc: 0.84 - Phosphorus: 240.2 - Fatty_acids_polyunsaturated: 0.38 - Fatty_acids_monounsaturated: 0.45 - Alcohol: 0.0 - Starch: 22.02 - Sodium: 36.4 - Water: 134.18 - Protein: 27.77 - Cholesterol: 63.2 PREVIOUS MEAL #6: Food Item: Proteinmilch Ananas Time: 2025-05-12 14:50 nutrients: - Energy_kcal: 290.0 - Fat: 1.5 - Fatty_acids_saturated: 1.0 - Carbohydrates: 22.5 - Sugar: 22.0 - Salt: 1.0 - Fiber: 1.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 35.0 - Cholesterol: 0.0 PREVIOUS MEAL #7: Food Item: Chicken with curry and coconut milk sauce, Rice, basmati, cooked, Mayonnaise Time: 2025-05-12 11:59 nutrients: - Energy_kcal: 660.4 - Fat: 45.82 - Fatty_acids_saturated: 10.65 - Carbohydrates: 35.15 - Sugar: 2.88 - Salt: 2.33 - Fiber: 3.18 - Calcium: 51.6 - Iron: 1.36 - Zinc: 1.35 - Phosphorus: 254.8 - Fatty_acids_polyunsaturated: 19.57 - Fatty_acids_monounsaturated: 13.27 - Alcohol: 0.0 - Starch: 31.29 - Sodium: 932.6 - Water: 226.56 - Protein: 22.81 - Cholesterol: 127.14 PREVIOUS MEAL #8: Food Item: Espresso with coffee cream, no sugar added, Croissant (average) Time: 2025-05-12 09:31 nutrients: - Energy_kcal: 256.8 - Fat: 14.7 - Fatty_acids_saturated: 8.58 - Carbohydrates: 25.14 - Sugar: 2.04 - Salt: 0.72 - Fiber: 2.1 - Calcium: 33.6 - Iron: 0.6 - Zinc: 0.6 - Phosphorus: 76.2 - Fatty_acids_polyunsaturated: 0.9 - Fatty_acids_monounsaturated: 3.42 - Alcohol: 0.0 - Starch: 22.8 - Sodium: 284.4 - Water: 73.5 - Protein: 4.98 - Cholesterol: 46.8 PREVIOUS MEAL #9: Food Item: Skyr Time: 2025-05-12 08:50 nutrients: - Energy_kcal: 85.5 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 6.3 - Sugar: 4.8 - Salt: 0.12 - Fiber: 0.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 15.0 - Cholesterol: 0.0 PREVIOUS MEAL #10: Food Item: Beer, lager, Fish product, anchovy in oil, drained, Mozzarella, Tomato sauce, Pizza doug ( with olive oil), raw Time: 2025-05-11 19:52 nutrients: - Energy_kcal: 889.3 - Fat: 28.93 - Fatty_acids_saturated: 13.13 - Carbohydrates: 92.88 - Sugar: 11.61 - Salt: 4.58 - Fiber: 5.12 - Calcium: 424.07 - Iron: 2.35 - Zinc: 3.9 - Phosphorus: 560.3 - Fatty_acids_polyunsaturated: 2.46 - Fatty_acids_monounsaturated: 11.19 - Alcohol: 13.2 - Starch: 77.48 - Sodium: 1845.77 - Water: 520.58 - Protein: 39.15 - Cholesterol: 57.4",
                "MOST RECENT MEAL: Food Item: Salmon, smoked, Sandwich loaf, wholemeal, Braided white bread made with butter Time: 2025-05-13 12:30 nutrients: - Energy_kcal: 686.45 - Fat: 21.59 - Fatty_acids_saturated: 10.78 - Carbohydrates: 94.77 - Sugar: 7.76 - Salt: 3.98 - Fiber: 6.89 - Calcium: 160.6 - Iron: 1.71 - Zinc: 1.66 - Phosphorus: 308.5 - Fatty_acids_polyunsaturated: 2.49 - Fatty_acids_monounsaturated: 5.31 - Alcohol: 0.12 - Starch: 84.42 - Sodium: 1582.6 - Water: 85.61 - Protein: 24.41 - Cholesterol: 74.0 PREVIOUS MEAL #1: Food Item: Chiefs - Protein bar, crispy cookie Time: 2025-05-13 10:49 nutrients: - Energy_kcal: 210.65 - Fat: 8.25 - Fatty_acids_saturated: 5.0 - Carbohydrates: 20.35 - Sugar: 1.59 - Salt: 0.45 - Fiber: 0.94 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 20.35 - Cholesterol: 0.0 PREVIOUS MEAL #2: Food Item: Mineral still water (Evian), bottled, lightly mineralized, Pineapple, pulp, raw Time: 2025-05-13 09:52 nutrients: - Energy_kcal: 48.96 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 10.53 - Sugar: 9.45 - Salt: 0.0 - Fiber: 1.08 - Calcium: 13.2 - Iron: 0.15 - Zinc: 0.07 - Phosphorus: 7.29 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.49 - Water: 151.95 - Protein: 0.0 - Cholesterol: 0.0 PREVIOUS MEAL #3: Food Item: Skyr, Meal replacement low calorie, in powder, reconstituted with skimmed milk, milkshake type, Beer, lager Time: 2025-05-12 22:50 nutrients: - Energy_kcal: 472.5 - Fat: 6.84 - Fatty_acids_saturated: 0.84 - Carbohydrates: 46.2 - Sugar: 41.4 - Salt: 0.69 - Fiber: 1.89 - Calcium: 526.5 - Iron: 5.73 - Zinc: 9.39 - Phosphorus: 471.0 - Fatty_acids_polyunsaturated: 3.69 - Fatty_acids_monounsaturated: 1.98 - Alcohol: 12.0 - Starch: 0.6 - Sodium: 237.6 - Water: 515.7 - Protein: 35.07 - Cholesterol: 30.0 PREVIOUS MEAL #4: Food Item: Broccoli, raw, Rice-maize based pasta, gluten-free, boiled, Fish, breaded, fried Time: 2025-05-12 19:12 nutrients: - Energy_kcal: 223.9 - Fat: 1.42 - Fatty_acids_saturated: 0.47 - Carbohydrates: 23.56 - Sugar: 0.95 - Salt: 0.08 - Fiber: 2.38 - Calcium: 32.04 - Iron: 0.85 - Zinc: 0.84 - Phosphorus: 240.2 - Fatty_acids_polyunsaturated: 0.38 - Fatty_acids_monounsaturated: 0.45 - Alcohol: 0.0 - Starch: 22.02 - Sodium: 36.4 - Water: 134.18 - Protein: 27.77 - Cholesterol: 63.2 PREVIOUS MEAL #5: Food Item: Proteinmilch Ananas Time: 2025-05-12 14:50 nutrients: - Energy_kcal: 290.0 - Fat: 1.5 - Fatty_acids_saturated: 1.0 - Carbohydrates: 22.5 - Sugar: 22.0 - Salt: 1.0 - Fiber: 1.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 35.0 - Cholesterol: 0.0 PREVIOUS MEAL #6: Food Item: Chicken with curry and coconut milk sauce, Rice, basmati, cooked, Mayonnaise Time: 2025-05-12 11:59 nutrients: - Energy_kcal: 660.4 - Fat: 45.82 - Fatty_acids_saturated: 10.65 - Carbohydrates: 35.15 - Sugar: 2.88 - Salt: 2.33 - Fiber: 3.18 - Calcium: 51.6 - Iron: 1.36 - Zinc: 1.35 - Phosphorus: 254.8 - Fatty_acids_polyunsaturated: 19.57 - Fatty_acids_monounsaturated: 13.27 - Alcohol: 0.0 - Starch: 31.29 - Sodium: 932.6 - Water: 226.56 - Protein: 22.81 - Cholesterol: 127.14 PREVIOUS MEAL #7: Food Item: Espresso with coffee cream, no sugar added, Croissant (average) Time: 2025-05-12 09:31 nutrients: - Energy_kcal: 256.8 - Fat: 14.7 - Fatty_acids_saturated: 8.58 - Carbohydrates: 25.14 - Sugar: 2.04 - Salt: 0.72 - Fiber: 2.1 - Calcium: 33.6 - Iron: 0.6 - Zinc: 0.6 - Phosphorus: 76.2 - Fatty_acids_polyunsaturated: 0.9 - Fatty_acids_monounsaturated: 3.42 - Alcohol: 0.0 - Starch: 22.8 - Sodium: 284.4 - Water: 73.5 - Protein: 4.98 - Cholesterol: 46.8 PREVIOUS MEAL #8: Food Item: Skyr Time: 2025-05-12 08:50 nutrients: - Energy_kcal: 85.5 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 6.3 - Sugar: 4.8 - Salt: 0.12 - Fiber: 0.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 15.0 - Cholesterol: 0.0 PREVIOUS MEAL #9: Food Item: Beer, lager, Fish product, anchovy in oil, drained, Mozzarella, Tomato sauce, Pizza doug ( with olive oil), raw Time: 2025-05-11 19:52 nutrients: - Energy_kcal: 889.3 - Fat: 28.93 - Fatty_acids_saturated: 13.13 - Carbohydrates: 92.88 - Sugar: 11.61 - Salt: 4.58 - Fiber: 5.12 - Calcium: 424.07 - Iron: 2.35 - Zinc: 3.9 - Phosphorus: 560.3 - Fatty_acids_polyunsaturated: 2.46 - Fatty_acids_monounsaturated: 11.19 - Alcohol: 13.2 - Starch: 77.48 - Sodium: 1845.77 - Water: 520.58 - Protein: 39.15 - Cholesterol: 57.4 PREVIOUS MEAL #10: Food Item: Skyr, Horseradish, fresh, Salmon, wild, raw, Butter of choice, Braided white bread made with butter Time: 2025-05-11 17:52 nutrients: - Energy_kcal: 538.13 - Fat: 24.32 - Fatty_acids_saturated: 11.23 - Carbohydrates: 44.23 - Sugar: 7.18 - Salt: 1.44 - Fiber: 2.43 - Calcium: 55.17 - Iron: 0.77 - Zinc: 0.86 - Phosphorus: 238.25 - Fatty_acids_polyunsaturated: 2.28 - Fatty_acids_monounsaturated: 6.66 - Alcohol: 0.0 - Starch: 33.92 - Sodium: 510.4 - Water: 73.57 - Protein: 34.84 - Cholesterol: 86.9",
                "MOST RECENT MEAL: Food Item: Skyr, Meal replacement low calorie, in powder, reconstituted with skimmed milk, milkshake type, Beer, lager Time: 2025-05-12 22:50 nutrients: - Energy_kcal: 472.5 - Fat: 6.84 - Fatty_acids_saturated: 0.84 - Carbohydrates: 46.2 - Sugar: 41.4 - Salt: 0.69 - Fiber: 1.89 - Calcium: 526.5 - Iron: 5.73 - Zinc: 9.39 - Phosphorus: 471.0 - Fatty_acids_polyunsaturated: 3.69 - Fatty_acids_monounsaturated: 1.98 - Alcohol: 12.0 - Starch: 0.6 - Sodium: 237.6 - Water: 515.7 - Protein: 35.07 - Cholesterol: 30.0 PREVIOUS MEAL #1: Food Item: Broccoli, raw, Rice-maize based pasta, gluten-free, boiled, Fish, breaded, fried Time: 2025-05-12 19:12 nutrients: - Energy_kcal: 223.9 - Fat: 1.42 - Fatty_acids_saturated: 0.47 - Carbohydrates: 23.56 - Sugar: 0.95 - Salt: 0.08 - Fiber: 2.38 - Calcium: 32.04 - Iron: 0.85 - Zinc: 0.84 - Phosphorus: 240.2 - Fatty_acids_polyunsaturated: 0.38 - Fatty_acids_monounsaturated: 0.45 - Alcohol: 0.0 - Starch: 22.02 - Sodium: 36.4 - Water: 134.18 - Protein: 27.77 - Cholesterol: 63.2 PREVIOUS MEAL #2: Food Item: Proteinmilch Ananas Time: 2025-05-12 14:50 nutrients: - Energy_kcal: 290.0 - Fat: 1.5 - Fatty_acids_saturated: 1.0 - Carbohydrates: 22.5 - Sugar: 22.0 - Salt: 1.0 - Fiber: 1.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 35.0 - Cholesterol: 0.0 PREVIOUS MEAL #3: Food Item: Chicken with curry and coconut milk sauce, Rice, basmati, cooked, Mayonnaise Time: 2025-05-12 11:59 nutrients: - Energy_kcal: 660.4 - Fat: 45.82 - Fatty_acids_saturated: 10.65 - Carbohydrates: 35.15 - Sugar: 2.88 - Salt: 2.33 - Fiber: 3.18 - Calcium: 51.6 - Iron: 1.36 - Zinc: 1.35 - Phosphorus: 254.8 - Fatty_acids_polyunsaturated: 19.57 - Fatty_acids_monounsaturated: 13.27 - Alcohol: 0.0 - Starch: 31.29 - Sodium: 932.6 - Water: 226.56 - Protein: 22.81 - Cholesterol: 127.14 PREVIOUS MEAL #4: Food Item: Espresso with coffee cream, no sugar added, Croissant (average) Time: 2025-05-12 09:31 nutrients: - Energy_kcal: 256.8 - Fat: 14.7 - Fatty_acids_saturated: 8.58 - Carbohydrates: 25.14 - Sugar: 2.04 - Salt: 0.72 - Fiber: 2.1 - Calcium: 33.6 - Iron: 0.6 - Zinc: 0.6 - Phosphorus: 76.2 - Fatty_acids_polyunsaturated: 0.9 - Fatty_acids_monounsaturated: 3.42 - Alcohol: 0.0 - Starch: 22.8 - Sodium: 284.4 - Water: 73.5 - Protein: 4.98 - Cholesterol: 46.8 PREVIOUS MEAL #5: Food Item: Skyr Time: 2025-05-12 08:50 nutrients: - Energy_kcal: 85.5 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 6.3 - Sugar: 4.8 - Salt: 0.12 - Fiber: 0.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 15.0 - Cholesterol: 0.0 PREVIOUS MEAL #6: Food Item: Beer, lager, Fish product, anchovy in oil, drained, Mozzarella, Tomato sauce, Pizza doug ( with olive oil), raw Time: 2025-05-11 19:52 nutrients: - Energy_kcal: 889.3 - Fat: 28.93 - Fatty_acids_saturated: 13.13 - Carbohydrates: 92.88 - Sugar: 11.61 - Salt: 4.58 - Fiber: 5.12 - Calcium: 424.07 - Iron: 2.35 - Zinc: 3.9 - Phosphorus: 560.3 - Fatty_acids_polyunsaturated: 2.46 - Fatty_acids_monounsaturated: 11.19 - Alcohol: 13.2 - Starch: 77.48 - Sodium: 1845.77 - Water: 520.58 - Protein: 39.15 - Cholesterol: 57.4 PREVIOUS MEAL #7: Food Item: Skyr, Horseradish, fresh, Salmon, wild, raw, Butter of choice, Braided white bread made with butter Time: 2025-05-11 17:52 nutrients: - Energy_kcal: 538.13 - Fat: 24.32 - Fatty_acids_saturated: 11.23 - Carbohydrates: 44.23 - Sugar: 7.18 - Salt: 1.44 - Fiber: 2.43 - Calcium: 55.17 - Iron: 0.77 - Zinc: 0.86 - Phosphorus: 238.25 - Fatty_acids_polyunsaturated: 2.28 - Fatty_acids_monounsaturated: 6.66 - Alcohol: 0.0 - Starch: 33.92 - Sodium: 510.4 - Water: 73.57 - Protein: 34.84 - Cholesterol: 86.9 PREVIOUS MEAL #8: Food Item: Honey, from flowers, Butter of choice, Braided white bread made with butter, Isey Skyr Protein Drink Strawberry-Banana, Skimmed milk powder Time: 2025-05-11 13:27 nutrients: - Energy_kcal: 479.78 - Fat: 15.78 - Fatty_acids_saturated: 8.91 - Carbohydrates: 56.44 - Sugar: 29.25 - Salt: 1.35 - Fiber: 1.26 - Calcium: 173.85 - Iron: 0.46 - Zinc: 0.87 - Phosphorus: 170.95 - Fatty_acids_polyunsaturated: 0.92 - Fatty_acids_monounsaturated: 3.51 - Alcohol: 0.0 - Starch: 25.44 - Sodium: 416.65 - Water: 24.53 - Protein: 21.9 - Cholesterol: 49.03 PREVIOUS MEAL #9: Food Item: Chicken egg, whole, hard boiled Time: 2025-05-11 09:18 nutrients: - Energy_kcal: 85.8 - Fat: 6.0 - Fatty_acids_saturated: 1.59 - Carbohydrates: 0.17 - Sugar: 0.17 - Salt: 0.22 - Fiber: 0.0 - Calcium: 25.3 - Iron: 0.94 - Zinc: 0.66 - Phosphorus: 99.0 - Fatty_acids_polyunsaturated: 0.99 - Fatty_acids_monounsaturated: 2.25 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 88.0 - Water: 41.09 - Protein: 7.7 - Cholesterol: 213.95 PREVIOUS MEAL #10: Food Item: Chocolate soft cake, prepacked Time: 2025-05-10 22:00 nutrients: - Energy_kcal: 195.1 - Fat: 10.8 - Fatty_acids_saturated: 3.55 - Carbohydrates: 21.69 - Sugar: 15.03 - Salt: 0.14 - Fiber: 1.15 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 2.07 - Fatty_acids_monounsaturated: 4.68 - Alcohol: 0.0 - Starch: 4.05 - Sodium: 54.0 - Water: 7.51 - Protein: 2.79 - Cholesterol: 57.15",
                "MOST RECENT MEAL: Food Item: Espresso with coffee cream, no sugar added, Croissant (average) Time: 2025-05-12 09:31 nutrients: - Energy_kcal: 256.8 - Fat: 14.7 - Fatty_acids_saturated: 8.58 - Carbohydrates: 25.14 - Sugar: 2.04 - Salt: 0.72 - Fiber: 2.1 - Calcium: 33.6 - Iron: 0.6 - Zinc: 0.6 - Phosphorus: 76.2 - Fatty_acids_polyunsaturated: 0.9 - Fatty_acids_monounsaturated: 3.42 - Alcohol: 0.0 - Starch: 22.8 - Sodium: 284.4 - Water: 73.5 - Protein: 4.98 - Cholesterol: 46.8 PREVIOUS MEAL #1: Food Item: Skyr, Horseradish, fresh, Salmon, wild, raw, Butter of choice, Braided white bread made with butter Time: 2025-05-11 17:52 nutrients: - Energy_kcal: 538.13 - Fat: 24.32 - Fatty_acids_saturated: 11.23 - Carbohydrates: 44.23 - Sugar: 7.18 - Salt: 1.44 - Fiber: 2.43 - Calcium: 55.17 - Iron: 0.77 - Zinc: 0.86 - Phosphorus: 238.25 - Fatty_acids_polyunsaturated: 2.28 - Fatty_acids_monounsaturated: 6.66 - Alcohol: 0.0 - Starch: 33.92 - Sodium: 510.4 - Water: 73.57 - Protein: 34.84 - Cholesterol: 86.9 PREVIOUS MEAL #2: Food Item: Honey, from flowers, Butter of choice, Braided white bread made with butter, Isey Skyr Protein Drink Strawberry-Banana, Skimmed milk powder Time: 2025-05-11 13:27 nutrients: - Energy_kcal: 479.78 - Fat: 15.78 - Fatty_acids_saturated: 8.91 - Carbohydrates: 56.44 - Sugar: 29.25 - Salt: 1.35 - Fiber: 1.26 - Calcium: 173.85 - Iron: 0.46 - Zinc: 0.87 - Phosphorus: 170.95 - Fatty_acids_polyunsaturated: 0.92 - Fatty_acids_monounsaturated: 3.51 - Alcohol: 0.0 - Starch: 25.44 - Sodium: 416.65 - Water: 24.53 - Protein: 21.9 - Cholesterol: 49.03 PREVIOUS MEAL #3: Food Item: Chicken egg, whole, hard boiled Time: 2025-05-11 09:18 nutrients: - Energy_kcal: 85.8 - Fat: 6.0 - Fatty_acids_saturated: 1.59 - Carbohydrates: 0.17 - Sugar: 0.17 - Salt: 0.22 - Fiber: 0.0 - Calcium: 25.3 - Iron: 0.94 - Zinc: 0.66 - Phosphorus: 99.0 - Fatty_acids_polyunsaturated: 0.99 - Fatty_acids_monounsaturated: 2.25 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 88.0 - Water: 41.09 - Protein: 7.7 - Cholesterol: 213.95 PREVIOUS MEAL #4: Food Item: Chocolate soft cake, prepacked Time: 2025-05-10 22:00 nutrients: - Energy_kcal: 195.1 - Fat: 10.8 - Fatty_acids_saturated: 3.55 - Carbohydrates: 21.69 - Sugar: 15.03 - Salt: 0.14 - Fiber: 1.15 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 2.07 - Fatty_acids_monounsaturated: 4.68 - Alcohol: 0.0 - Starch: 4.05 - Sodium: 54.0 - Water: 7.51 - Protein: 2.79 - Cholesterol: 57.15 PREVIOUS MEAL #5: Food Item: Broccoli, steamed, Mushroom sauce, thickened, Beef, steak or beef steak, grilled, Pasta with egg, dry, Mushroom sauce, thickened, Veal,steak, pan fried (without addition of fat and salt), Pasta with egg, dry, Olive oil, Lettuce, raw Time: 2025-05-10 19:43 nutrients: - Energy_kcal: 988.94 - Fat: 21.7 - Fatty_acids_saturated: 7.9 - Carbohydrates: 127.89 - Sugar: 10.71 - Salt: 0.89 - Fiber: 7.65 - Calcium: 191.53 - Iron: 8.53 - Zinc: 9.08 - Phosphorus: 541.74 - Fatty_acids_polyunsaturated: 2.92 - Fatty_acids_monounsaturated: 7.52 - Alcohol: 0.0 - Starch: 112.76 - Sodium: 377.61 - Water: 249.43 - Protein: 66.59 - Cholesterol: 309.51 PREVIOUS MEAL #6: Food Item: Hummus, Sausage, dried, Pommes Chips, Beer, lager Time: 2025-05-10 16:29 nutrients: - Energy_kcal: 537.3 - Fat: 28.57 - Fatty_acids_saturated: 6.3 - Carbohydrates: 30.23 - Sugar: 8.82 - Salt: 2.33 - Fiber: 3.87 - Calcium: 100.75 - Iron: 1.32 - Zinc: 0.83 - Phosphorus: 175.3 - Fatty_acids_polyunsaturated: 3.93 - Fatty_acids_monounsaturated: 16.9 - Alcohol: 13.2 - Starch: 18.14 - Sodium: 956.27 - Water: 345.59 - Protein: 14.96 - Cholesterol: 0.0 PREVIOUS MEAL #7: Food Item: Riegel high protein coconut almond bar Time: 2025-05-10 14:53 nutrients: - Energy_kcal: 132.65 - Fat: 5.25 - Fatty_acids_saturated: 2.59 - Carbohydrates: 7.35 - Sugar: 0.32 - Salt: 0.17 - Fiber: 1.72 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 66.15 - Water: 0.0 - Protein: 15.75 - Cholesterol: 0.0 PREVIOUS MEAL #8: Food Item: Isey Skyr Protein Drink Strawberry-Banana, Skimmed milk powder Time: 2025-05-10 13:30 nutrients: - Energy_kcal: 164.28 - Fat: 0.71 - Fatty_acids_saturated: 0.07 - Carbohydrates: 17.55 - Sugar: 16.05 - Salt: 0.45 - Fiber: 0.0 - Calcium: 141.9 - Iron: 0.09 - Zinc: 0.45 - Phosphorus: 112.2 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.02 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 61.6 - Water: 0.39 - Protein: 16.37 - Cholesterol: 0.33 PREVIOUS MEAL #9: Food Item: Falafel (deep fried in HOLL rapeseed oil), Chicken egg, whole, hard boiled, Butterhead lettuce, raw, Olive oil Time: 2025-05-10 13:13 nutrients: - Energy_kcal: 267.3 - Fat: 18.13 - Fatty_acids_saturated: 2.83 - Carbohydrates: 12.6 - Sugar: 1.6 - Salt: 0.51 - Fiber: 3.84 - Calcium: 60.5 - Iron: 1.82 - Zinc: 1.08 - Phosphorus: 158.88 - Fatty_acids_polyunsaturated: 2.44 - Fatty_acids_monounsaturated: 11.07 - Alcohol: 0.0 - Starch: 10.76 - Sodium: 216.47 - Water: 117.39 - Protein: 11.52 - Cholesterol: 194.5 PREVIOUS MEAL #10: Food Item: Biscuit (cookie), with chocolate drops Time: 2025-05-10 11:54 nutrients: - Energy_kcal: 197.99 - Fat: 10.36 - Fatty_acids_saturated: 5.16 - Carbohydrates: 23.6 - Sugar: 12.6 - Salt: 0.4 - Fiber: 1.35 - Calcium: 12.2 - Iron: 1.37 - Zinc: 0.48 - Phosphorus: 61.2 - Fatty_acids_polyunsaturated: 0.93 - Fatty_acids_monounsaturated: 3.84 - Alcohol: 0.0 - Starch: 11.0 - Sodium: 161.2 - Water: 1.38 - Protein: 2.59 - Cholesterol: 6.92",
                "MOST RECENT MEAL: Food Item: Chicken egg, whole, hard boiled Time: 2025-05-11 09:18 nutrients: - Energy_kcal: 85.8 - Fat: 6.0 - Fatty_acids_saturated: 1.59 - Carbohydrates: 0.17 - Sugar: 0.17 - Salt: 0.22 - Fiber: 0.0 - Calcium: 25.3 - Iron: 0.94 - Zinc: 0.66 - Phosphorus: 99.0 - Fatty_acids_polyunsaturated: 0.99 - Fatty_acids_monounsaturated: 2.25 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 88.0 - Water: 41.09 - Protein: 7.7 - Cholesterol: 213.95 PREVIOUS MEAL #1: Food Item: Chocolate soft cake, prepacked Time: 2025-05-10 22:00 nutrients: - Energy_kcal: 195.1 - Fat: 10.8 - Fatty_acids_saturated: 3.55 - Carbohydrates: 21.69 - Sugar: 15.03 - Salt: 0.14 - Fiber: 1.15 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 2.07 - Fatty_acids_monounsaturated: 4.68 - Alcohol: 0.0 - Starch: 4.05 - Sodium: 54.0 - Water: 7.51 - Protein: 2.79 - Cholesterol: 57.15 PREVIOUS MEAL #2: Food Item: Broccoli, steamed, Mushroom sauce, thickened, Beef, steak or beef steak, grilled, Pasta with egg, dry, Mushroom sauce, thickened, Veal,steak, pan fried (without addition of fat and salt), Pasta with egg, dry, Olive oil, Lettuce, raw Time: 2025-05-10 19:43 nutrients: - Energy_kcal: 988.94 - Fat: 21.7 - Fatty_acids_saturated: 7.9 - Carbohydrates: 127.89 - Sugar: 10.71 - Salt: 0.89 - Fiber: 7.65 - Calcium: 191.53 - Iron: 8.53 - Zinc: 9.08 - Phosphorus: 541.74 - Fatty_acids_polyunsaturated: 2.92 - Fatty_acids_monounsaturated: 7.52 - Alcohol: 0.0 - Starch: 112.76 - Sodium: 377.61 - Water: 249.43 - Protein: 66.59 - Cholesterol: 309.51 PREVIOUS MEAL #3: Food Item: Riegel high protein coconut almond bar Time: 2025-05-10 14:53 nutrients: - Energy_kcal: 132.65 - Fat: 5.25 - Fatty_acids_saturated: 2.59 - Carbohydrates: 7.35 - Sugar: 0.32 - Salt: 0.17 - Fiber: 1.72 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 66.15 - Water: 0.0 - Protein: 15.75 - Cholesterol: 0.0 PREVIOUS MEAL #4: Food Item: Falafel (deep fried in HOLL rapeseed oil), Chicken egg, whole, hard boiled, Butterhead lettuce, raw, Olive oil Time: 2025-05-10 13:13 nutrients: - Energy_kcal: 267.3 - Fat: 18.13 - Fatty_acids_saturated: 2.83 - Carbohydrates: 12.6 - Sugar: 1.6 - Salt: 0.51 - Fiber: 3.84 - Calcium: 60.5 - Iron: 1.82 - Zinc: 1.08 - Phosphorus: 158.88 - Fatty_acids_polyunsaturated: 2.44 - Fatty_acids_monounsaturated: 11.07 - Alcohol: 0.0 - Starch: 10.76 - Sodium: 216.47 - Water: 117.39 - Protein: 11.52 - Cholesterol: 194.5 PREVIOUS MEAL #5: Food Item: Biscuit (cookie), with chocolate drops Time: 2025-05-10 11:54 nutrients: - Energy_kcal: 197.99 - Fat: 10.36 - Fatty_acids_saturated: 5.16 - Carbohydrates: 23.6 - Sugar: 12.6 - Salt: 0.4 - Fiber: 1.35 - Calcium: 12.2 - Iron: 1.37 - Zinc: 0.48 - Phosphorus: 61.2 - Fatty_acids_polyunsaturated: 0.93 - Fatty_acids_monounsaturated: 3.84 - Alcohol: 0.0 - Starch: 11.0 - Sodium: 161.2 - Water: 1.38 - Protein: 2.59 - Cholesterol: 6.92 PREVIOUS MEAL #6: Food Item: Kellogg's Special K - Protein Nuts, clusters and seeds, Naturaplan : Drink : Lait partiellement écrémé, Skyr Time: 2025-05-10 08:27 nutrients: - Energy_kcal: 473.9 - Fat: 10.3 - Fatty_acids_saturated: 3.42 - Carbohydrates: 61.6 - Sugar: 23.0 - Salt: 0.83 - Fiber: 5.81 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 30.5 - Cholesterol: 0.0 PREVIOUS MEAL #7: Food Item: Spinach quiche, baked (with cake dough), Wholewheat bread, Trout, farmed, smoked Time: 2025-05-09 19:17 nutrients: - Energy_kcal: 612.75 - Fat: 31.49 - Fatty_acids_saturated: 12.47 - Carbohydrates: 36.76 - Sugar: 2.04 - Salt: 6.45 - Fiber: 5.48 - Calcium: 163.38 - Iron: 3.92 - Zinc: 2.71 - Phosphorus: 504.75 - Fatty_acids_polyunsaturated: 4.89 - Fatty_acids_monounsaturated: 9.02 - Alcohol: 0.0 - Starch: 32.42 - Sodium: 2582.5 - Water: 217.79 - Protein: 42.7 - Cholesterol: 137.28 PREVIOUS MEAL #8: Food Item: Skyr Time: 2025-05-09 18:49 nutrients: - Energy_kcal: 85.5 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 6.3 - Sugar: 4.8 - Salt: 0.12 - Fiber: 0.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 15.0 - Cholesterol: 0.0 PREVIOUS MEAL #9: Food Item: Riegel high protein coconut almond bar Time: 2025-05-09 16:50 nutrients: - Energy_kcal: 132.65 - Fat: 5.25 - Fatty_acids_saturated: 2.59 - Carbohydrates: 7.35 - Sugar: 0.32 - Salt: 0.17 - Fiber: 1.72 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 66.15 - Water: 0.0 - Protein: 15.75 - Cholesterol: 0.0 PREVIOUS MEAL #10: Food Item: Amaretti (Almond cookie), Espresso with coffee cream, no sugar added Time: 2025-05-09 12:52 nutrients: - Energy_kcal: 54.8 - Fat: 2.05 - Fatty_acids_saturated: 0.55 - Carbohydrates: 7.91 - Sugar: 7.85 - Salt: 0.01 - Fiber: 0.29 - Calcium: 13.8 - Iron: 0.09 - Zinc: 0.08 - Phosphorus: 18.1 - Fatty_acids_polyunsaturated: 0.31 - Fatty_acids_monounsaturated: 0.96 - Alcohol: 0.0 - Starch: 0.01 - Sodium: 8.4 - Water: 28.83 - Protein: 1.03 - Cholesterol: 2.4",
                "MOST RECENT MEAL: Food Item: Spinach quiche, baked (with cake dough), Wholewheat bread, Trout, farmed, smoked Time: 2025-05-09 19:17 nutrients: - Energy_kcal: 612.75 - Fat: 31.49 - Fatty_acids_saturated: 12.47 - Carbohydrates: 36.76 - Sugar: 2.04 - Salt: 6.45 - Fiber: 5.48 - Calcium: 163.38 - Iron: 3.92 - Zinc: 2.71 - Phosphorus: 504.75 - Fatty_acids_polyunsaturated: 4.89 - Fatty_acids_monounsaturated: 9.02 - Alcohol: 0.0 - Starch: 32.42 - Sodium: 2582.5 - Water: 217.79 - Protein: 42.7 - Cholesterol: 137.28 PREVIOUS MEAL #1: Food Item: Skyr Time: 2025-05-09 18:49 nutrients: - Energy_kcal: 85.5 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 6.3 - Sugar: 4.8 - Salt: 0.12 - Fiber: 0.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 15.0 - Cholesterol: 0.0 PREVIOUS MEAL #2: Food Item: Riegel high protein coconut almond bar Time: 2025-05-09 16:50 nutrients: - Energy_kcal: 132.65 - Fat: 5.25 - Fatty_acids_saturated: 2.59 - Carbohydrates: 7.35 - Sugar: 0.32 - Salt: 0.17 - Fiber: 1.72 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 66.15 - Water: 0.0 - Protein: 15.75 - Cholesterol: 0.0 PREVIOUS MEAL #3: Food Item: Amaretti (Almond cookie), Espresso with coffee cream, no sugar added Time: 2025-05-09 12:52 nutrients: - Energy_kcal: 54.8 - Fat: 2.05 - Fatty_acids_saturated: 0.55 - Carbohydrates: 7.91 - Sugar: 7.85 - Salt: 0.01 - Fiber: 0.29 - Calcium: 13.8 - Iron: 0.09 - Zinc: 0.08 - Phosphorus: 18.1 - Fatty_acids_polyunsaturated: 0.31 - Fatty_acids_monounsaturated: 0.96 - Alcohol: 0.0 - Starch: 0.01 - Sodium: 8.4 - Water: 28.83 - Protein: 1.03 - Cholesterol: 2.4 PREVIOUS MEAL #4: Food Item: Pepper, sweet, red, sautéed/pan-fried, without fat, Quinoa, cooked (without addition of salt and fat), Gravy, brown, Meat loaf, baked in the oven Time: 2025-05-09 12:30 nutrients: - Energy_kcal: 399.59 - Fat: 13.99 - Fatty_acids_saturated: 3.53 - Carbohydrates: 36.57 - Sugar: 6.46 - Salt: 4.22 - Fiber: 4.28 - Calcium: 58.9 - Iron: 4.81 - Zinc: 5.04 - Phosphorus: 312.5 - Fatty_acids_polyunsaturated: 2.19 - Fatty_acids_monounsaturated: 7.18 - Alcohol: 0.0 - Starch: 29.24 - Sodium: 1697.97 - Water: 264.7 - Protein: 29.13 - Cholesterol: 102.0 PREVIOUS MEAL #5: Food Item: Salad dressing, french style, Leafy salad (average), raw Time: 2025-05-09 12:08 nutrients: - Energy_kcal: 63.5 - Fat: 5.76 - Fatty_acids_saturated: 0.63 - Carbohydrates: 1.48 - Sugar: 1.4 - Salt: 0.3 - Fiber: 0.89 - Calcium: 27.2 - Iron: 0.47 - Zinc: 0.19 - Phosphorus: 23.9 - Fatty_acids_polyunsaturated: 2.92 - Fatty_acids_monounsaturated: 1.83 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 130.0 - Water: 55.4 - Protein: 1.01 - Cholesterol: 11.25 PREVIOUS MEAL #6: Food Item: Kellogg's Special K - Protein Nuts, clusters and seeds, Naturaplan : Drink : Lait partiellement écrémé, Skyr Time: 2025-05-09 08:15 nutrients: - Energy_kcal: 473.9 - Fat: 10.3 - Fatty_acids_saturated: 3.42 - Carbohydrates: 61.6 - Sugar: 23.0 - Salt: 0.83 - Fiber: 5.81 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 0.0 - Protein: 30.5 - Cholesterol: 0.0 PREVIOUS MEAL #7: Food Item: Green bean, dried, soaked, steamed (without addition of salt), Cordon bleu from veal schnitzel, fried Time: 2025-05-08 19:56 nutrients: - Energy_kcal: 350.7 - Fat: 13.65 - Fatty_acids_saturated: 3.81 - Carbohydrates: 21.81 - Sugar: 5.76 - Salt: 1.08 - Fiber: 6.39 - Calcium: 258.0 - Iron: 4.02 - Zinc: 3.54 - Phosphorus: 401.1 - Fatty_acids_polyunsaturated: 1.71 - Fatty_acids_monounsaturated: 6.72 - Alcohol: 0.0 - Starch: 16.14 - Sodium: 425.31 - Water: 136.2 - Protein: 32.07 - Cholesterol: 115.2 PREVIOUS MEAL #8: Food Item: Emmi Energy Milk High Protein Erdbeer Time: 2025-05-08 19:10 nutrients: - Energy_kcal: 217.8 - Fat: 4.95 - Fatty_acids_saturated: 2.97 - Carbohydrates: 16.17 - Sugar: 15.84 - Salt: 0.89 - Fiber: 0.0 - Calcium: 0.0 - Iron: 0.0 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 356.4 - Water: 0.0 - Protein: 26.07 - Cholesterol: 0.0 PREVIOUS MEAL #9: Food Item: Parma dry-cured ham Time: 2025-05-08 18:36 nutrients: - Energy_kcal: 210.8 - Fat: 13.09 - Fatty_acids_saturated: 4.61 - Carbohydrates: 0.26 - Sugar: 0.26 - Salt: 4.7 - Fiber: 0.0 - Calcium: 13.6 - Iron: 0.59 - Zinc: 0.0 - Phosphorus: 0.0 - Fatty_acids_polyunsaturated: 1.3 - Fatty_acids_monounsaturated: 6.54 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 1853.0 - Water: 44.37 - Protein: 23.12 - Cholesterol: 0.0 PREVIOUS MEAL #10: Food Item: Pineapple, pulp, raw Time: 2025-05-08 15:42 nutrients: - Energy_kcal: 87.04 - Fat: 0.0 - Fatty_acids_saturated: 0.0 - Carbohydrates: 18.72 - Sugar: 16.8 - Salt: 0.0 - Fiber: 1.92 - Calcium: 12.8 - Iron: 0.27 - Zinc: 0.13 - Phosphorus: 12.96 - Fatty_acids_polyunsaturated: 0.0 - Fatty_acids_monounsaturated: 0.0 - Alcohol: 0.0 - Starch: 0.0 - Sodium: 0.0 - Water: 136.8 - Protein: 0.0 - Cholesterol: 0.0"
                 ])), 
    "diet_info": list(reversed([
            "['Fat intake is within the recommended range at 29.83% of calories.', 'Carbohydrates intake is low at 41.47% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 24.28% of calories.', 'Fiber intake is low at 15.28g (recommended: 25-35g).', 'Sugar intake is high at 60.23g (recommended: 0-50g).', 'Salt intake is high at 6.47g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 31.44% of calories.', 'Carbohydrates intake is low at 40.14% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 23.53% of calories.', 'Fiber intake is low at 15.26g (recommended: 25-35g).', 'Sugar intake is high at 61.50g (recommended: 0-50g).', 'Salt intake is high at 6.46g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 31.43% of calories.', 'Carbohydrates intake is low at 39.68% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 23.77% of calories.', 'Fiber intake is low at 15.57g (recommended: 25-35g).', 'Sugar intake is high at 63.42g (recommended: 0-50g).', 'Salt intake is high at 6.82g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 31.78% of calories.', 'Carbohydrates intake is low at 39.16% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 23.89% of calories.', 'Fiber intake is low at 14.95g (recommended: 25-35g).', 'Sugar intake is high at 62.68g (recommended: 0-50g).', 'Salt intake is high at 6.64g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 32.31% of calories.', 'Carbohydrates intake is low at 38.45% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 24.82% of calories.', 'Fiber intake is low at 12.49g (recommended: 25-35g).', 'Sugar intake is within the recommended range at 47.50g.', 'Salt intake is high at 6.30g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 31.87% of calories.', 'Carbohydrates intake is low at 38.71% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 24.94% of calories.', 'Fiber intake is low at 13.82g (recommended: 25-35g).', 'Sugar intake is high at 53.67g (recommended: 0-50g).', 'Salt intake is high at 6.85g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 32.15% of calories.', 'Carbohydrates intake is low at 37.55% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 25.42% of calories.', 'Fiber intake is low at 14.63g (recommended: 25-35g).', 'Sugar intake is high at 59.48g (recommended: 0-50g).', 'Salt intake is high at 7.26g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 33.05% of calories.', 'Carbohydrates intake is low at 38.41% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 25.06% of calories.', 'Fiber intake is low at 12.37g (recommended: 25-35g).', 'Sugar intake is within the recommended range at 45.54g.', 'Salt intake is high at 5.79g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 31.61% of calories.', 'Carbohydrates intake is low at 39.30% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 26.77% of calories.', 'Fiber intake is low at 12.92g (recommended: 25-35g).', 'Sugar intake is within the recommended range at 41.98g.', 'Salt intake is high at 5.69g (recommended: 0-5g).']",
            "['Fat intake is within the recommended range at 31.55% of calories.', 'Carbohydrates intake is low at 37.26% of calories (recommended: 45-60% of calories).', 'Protein intake is within the recommended range at 28.36% of calories.', 'Fiber intake is low at 14.35g (recommended: 25-35g).', 'Sugar intake is within the recommended range at 48.82g.', 'Salt intake is high at 8.43g (recommended: 0-5g).']"
            ]))
, 
    "current_logging_streak": list(reversed([
                    "10 days",
                    "9 days",
                    "8 days",
                    "8 days",
                    "8 days",
                    "7 days",
                    "6 days",
                    "6 days",
                    "5 days",
                    "3 days",
                    ])),
    "tlr": list(reversed([
        "16 hours, 59 minutes",
        "6 hours, 59 minutes",
        "16 hours, 59 minutes",
        "6 hours, 59 minutes",
        "4 hours, 59 minutes",
        "11 hours, 59 minutes",
        "18 hours, 59 minutes",
        "23 hours, 59 minutes",
        "1 day, 4 hours, 59 minutes",
        "18 hours, 59 minutes"
        ])),
    "prefered_language": "french"
}

In [ ]:

def vanilla_with_context(meal_type,age,gender,diet_goal,recent_meals,diet_info, current_logging_streak,
                          tlr,previous_reminders,prefered_language) :

    final_prompt = REMINDER_SYSTEM_ROLE_VANILLA.format(meal_type = meal_type)
    final_prompt += USER_PROFILE.format(
                                    age= age, gender = gender,
                                    diet_goal = diet_goal, recent_meals = recent_meals,
                                    diet_information = diet_info,
                                    current_logging_streak = current_logging_streak,
                                    prefered_language = prefered_language
                                    )
    final_prompt += REMINDER_CONTEXT.format(
                                        tlr = tlr,
                                        previous_reminders = previous_reminders,
                                        prefered_language = prefered_language
                                    )

    additional_prompt = select_personalization_prompt(diet_info)
    if additional_prompt: 
        additional_guideline = additional_prompt.format(
            diet_information=diet_info,
            diet_goal=diet_goal,
            recent_meals=recent_meals
        )
        final_prompt += additional_guideline
    
    final_prompt += INFLEXIBLE_RULES.format(prefered_language = prefered_language)

    return final_prompt
    
def vanilla_with_context_and_few_shots(meal_type,age,gender,diet_goal,recent_meals,diet_info, current_logging_streak,
                          tlr,previous_reminders,prefered_language) :
    
    final_prompt = REMINDER_SYSTEM_ROLE_VANILLA.format(meal_type = meal_type)
    final_prompt += USER_PROFILE.format(
                                    age= age, gender = gender,
                                    diet_goal = diet_goal, recent_meals = recent_meals,
                                    diet_information = diet_info,
                                    current_logging_streak = current_logging_streak,
                                    prefered_language = prefered_language
                                    )
    final_prompt += REMINDER_CONTEXT.format(
                                        tlr = tlr,
                                        previous_reminders = previous_reminders,
                                        prefered_language = prefered_language
                                    )
    
    additional_prompt = select_personalization_prompt(diet_info)
    if additional_prompt: 
        additional_guideline = additional_prompt.format(
            diet_information=diet_info,
            diet_goal=diet_goal,
            recent_meals=recent_meals
        )
        final_prompt += additional_guideline
    
    final_prompt += FEW_SHOTS_PROMPTING.format(meal_type = meal_type)

    final_prompt += INFLEXIBLE_RULES.format(prefered_language = prefered_language)

    return final_prompt

def vanilla_with_context_and_cot(meal_type,age,gender,diet_goal,recent_meals,diet_info, current_logging_streak,
                          tlr,previous_reminders,prefered_language) :
    
    final_prompt = REMINDER_SYSTEM_ROLE_VANILLA.format(meal_type = meal_type)
    final_prompt += USER_PROFILE.format(
                                    age= age, gender = gender,
                                    diet_goal = diet_goal, recent_meals = recent_meals,
                                    diet_information = diet_info,
                                    current_logging_streak = current_logging_streak,
                                    prefered_language = prefered_language
                                    )
    final_prompt += REMINDER_CONTEXT.format(
                                        tlr = tlr,
                                        previous_reminders = previous_reminders,
                                        prefered_language = prefered_language
                                    )
    
    additional_prompt = select_personalization_prompt(diet_info)
    potential = ""
    if additional_prompt: 
        additional_guideline = additional_prompt.format(
            diet_information=diet_info,
            diet_goal=diet_goal,
            recent_meals=recent_meals
        )
        potential = add_cot(additional_prompt)
        final_prompt += additional_guideline


    
    final_prompt += COT_REASONING
    final_prompt += potential
    

    final_prompt += INFLEXIBLE_RULES.format(prefered_language = prefered_language)

    return final_prompt


def vanilla_with_context_few_shots_and_cot(meal_type,age,gender,diet_goal,recent_meals,diet_info, current_logging_streak,
                          tlr,previous_reminders,prefered_language) :
    
    final_prompt = REMINDER_SYSTEM_ROLE_VANILLA.format(meal_type = meal_type)
    final_prompt += USER_PROFILE.format(
                                    age= age, gender = gender,
                                    diet_goal = diet_goal, recent_meals = recent_meals,
                                    diet_information = diet_info,
                                    current_logging_streak = current_logging_streak,
                                    prefered_language = prefered_language
                                    )
    final_prompt += REMINDER_CONTEXT.format(
                                        tlr = tlr,
                                        previous_reminders = previous_reminders,
                                        prefered_language = prefered_language
                                    )
    
    additional_prompt = select_personalization_prompt(diet_info)
    potential = ""
    if additional_prompt: 
        additional_guideline = additional_prompt.format(
            diet_information=diet_info,
            diet_goal=diet_goal,
            recent_meals=recent_meals
        )
        potential = add_cot(additional_prompt)
        final_prompt += additional_guideline

    final_prompt += FEW_SHOTS_PROMPTING.format(meal_type = meal_type)
    
    final_prompt += COT_REASONING
    final_prompt += potential

    final_prompt += INFLEXIBLE_RULES.format(prefered_language = prefered_language)

    return final_prompt

def persona_with_context(meal_type,age,gender,diet_goal,recent_meals,diet_info, current_logging_streak,
                          tlr,previous_reminders,prefered_language) :
    
    final_prompt = REMINDER_SYSTEM_ROLE_VANILLA.format(meal_type = meal_type)
    final_prompt += SYSTEM_PERSONA
    final_prompt += USER_PROFILE.format(
                                    age= age, gender = gender,
                                    diet_goal = diet_goal, recent_meals = recent_meals,
                                    diet_information = diet_info,
                                    current_logging_streak = current_logging_streak,
                                    prefered_language = prefered_language
                                    )
    final_prompt += REMINDER_CONTEXT.format(
                                        tlr = tlr,
                                        previous_reminders = previous_reminders,
                                        prefered_language = prefered_language
                                    )

    additional_prompt = select_personalization_prompt(diet_info)
    if additional_prompt: 
        additional_guideline = additional_prompt.format(
            diet_information=diet_info,
            diet_goal=diet_goal,
            recent_meals=recent_meals
        )
        final_prompt += additional_guideline

    final_prompt += INFLEXIBLE_RULES.format(prefered_language = prefered_language)

    return final_prompt

def persona_with_context_and_few_shots(meal_type,age,gender,diet_goal,recent_meals,diet_info, current_logging_streak,
                          tlr,previous_reminders,prefered_language) :
    
    final_prompt = REMINDER_SYSTEM_ROLE_VANILLA.format(meal_type = meal_type)
    final_prompt += SYSTEM_PERSONA
    final_prompt += USER_PROFILE.format(
                                    age= age, gender = gender,
                                    diet_goal = diet_goal, recent_meals = recent_meals,
                                    diet_information = diet_info,
                                    current_logging_streak = current_logging_streak,
                                    prefered_language = prefered_language
                                    )
    final_prompt += REMINDER_CONTEXT.format(
                                        tlr = tlr,
                                        previous_reminders = previous_reminders,
                                        prefered_language = prefered_language
                                    )

    additional_prompt = select_personalization_prompt(diet_info)
    if additional_prompt: 
        additional_guideline = additional_prompt.format(
            diet_information=diet_info,
            diet_goal=diet_goal,
            recent_meals=recent_meals
        )
        final_prompt += additional_guideline

    final_prompt += FEW_SHOTS_PROMPTING.format(meal_type = meal_type)

    final_prompt += INFLEXIBLE_RULES.format(prefered_language = prefered_language)

    return final_prompt

def persona_with_context_and_cot(meal_type,age,gender,diet_goal,recent_meals,diet_info, current_logging_streak,
                          tlr,previous_reminders,prefered_language) :
    
    final_prompt = REMINDER_SYSTEM_ROLE_VANILLA.format(meal_type = meal_type)
    final_prompt += SYSTEM_PERSONA
    final_prompt += USER_PROFILE.format(
                                    age= age, gender = gender,
                                    diet_goal = diet_goal, recent_meals = recent_meals,
                                    diet_information = diet_info,
                                    current_logging_streak = current_logging_streak,
                                    prefered_language = prefered_language
                                    )
    final_prompt += REMINDER_CONTEXT.format(
                                        tlr = tlr,
                                        previous_reminders = previous_reminders,
                                        prefered_language = prefered_language
                                    )
    
    additional_prompt = select_personalization_prompt(diet_info)
    potential = ""
    if additional_prompt: 
        additional_guideline = additional_prompt.format(
            diet_information=diet_info,
            diet_goal=diet_goal,
            recent_meals=recent_meals
        )
        potential = add_cot(additional_prompt)
        final_prompt += additional_guideline


    
    final_prompt += COT_REASONING
    final_prompt += potential
    
    final_prompt += INFLEXIBLE_RULES.format(prefered_language = prefered_language)

    return final_prompt


def persona_with_context_few_shots_and_cot(meal_type,age,gender,diet_goal,recent_meals,diet_info, current_logging_streak,
                          tlr,previous_reminders,prefered_language) :
    
    final_prompt = REMINDER_SYSTEM_ROLE_VANILLA.format(meal_type = meal_type)
    final_prompt += SYSTEM_PERSONA
    final_prompt += USER_PROFILE.format(
                                    age= age, gender = gender,
                                    diet_goal = diet_goal, recent_meals = recent_meals,
                                    diet_information = diet_info,
                                    current_logging_streak = current_logging_streak,
                                    prefered_language = prefered_language
                                    )
    final_prompt += REMINDER_CONTEXT.format(
                                        tlr = tlr,
                                        previous_reminders = previous_reminders,
                                        prefered_language = prefered_language,
                                    )

    additional_prompt = select_personalization_prompt(diet_info)
    potential = ""
    if additional_prompt: 
        additional_guideline = additional_prompt.format(
            diet_information=diet_info,
            diet_goal=diet_goal,
            recent_meals=recent_meals
        )
        potential = add_cot(additional_prompt)
        final_prompt += additional_guideline

    final_prompt += FEW_SHOTS_PROMPTING.format(meal_type = meal_type)

    final_prompt += COT_REASONING
    final_prompt += potential


    final_prompt += INFLEXIBLE_RULES.format(prefered_language = prefered_language)

    return final_prompt

In [9]:
from src.prompt_engineering.generating_framework import PromptTestingFramework

client = OpenAI()
framework = PromptTestingFramework(client, models[0], user_data)
functions_list = [vanilla_with_context,vanilla_with_context_and_few_shots,vanilla_with_context_and_cot,vanilla_with_context_few_shots_and_cot,persona_with_context,persona_with_context_and_few_shots,persona_with_context_and_cot,persona_with_context_few_shots_and_cot]
#functions_list = [persona_with_context_few_shots_and_cot]

for func in functions_list :
    framework.register_prompt_function(func.__name__,func)

In [10]:
results = {}
for func in functions_list:
    print(f"Testing {func.__name__}...")
    results[func.__name__] = framework.generate_reminders(func.__name__, n=10)

Testing vanilla_with_context...


Generating vanilla_with_context reminders:   0%|          | 0/10 [00:00<?, ?it/s]

Testing vanilla_with_context_and_few_shots...


Generating vanilla_with_context_and_few_shots reminders:   0%|          | 0/10 [00:00<?, ?it/s]

Testing vanilla_with_context_and_cot...


...

Generating persona_with_context_and_few_shots reminders:   0%|          | 0/10 [00:00<?, ?it/s]

Testing persona_with_context_and_cot...


Generating persona_with_context_and_cot reminders:   0%|          | 0/10 [00:00<?, ?it/s]

Testing persona_with_context_few_shots_and_cot...


Generating persona_with_context_few_shots_and_cot reminders:   0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
import json
with open("generated_reminders.json", 'w') as f:
    json.dump(results, f, indent=2)

## Prompt engineering - Conversational Assistant

In [5]:
import datetime
current_datetime = datetime.datetime.now(
    
)

In [6]:
import os
import dotenv
import pandas as pd

from src.constants import *
from src.openai_client import *

open_ai_api_key = os.environ.get('OPENAI_API_KEY')
dotenv.load_dotenv()
from src.communication.messaging_service import SMSService
from src.communication.twilio_tool import NewTwilioConversationManager
from src.data_manager.meal import get_recent_meals_string_for_user

from src.config.config import create_app


app = create_app()
recent_meals = get_recent_meals_string_for_user("UEtQ3hZyv")

Creating database tables
Tables created successfully
2025-05-22 15:39:54,831 - myfoodrepo.scheduler - INFO - Scheduling reminders for all users...


2025-05-22 15:39:54,898 - myfoodrepo.scheduler - INFO - Job ID: reminder_7, Hour: *
2025-05-22 15:39:54,900 - myfoodrepo.scheduler - INFO - Job ID: reminder_9, Hour: *
2025-05-22 15:39:54,902 - myfoodrepo.scheduler - INFO - Job ID: reminder_8, Hour: *
2025-05-22 15:39:54,903 - myfoodrepo.scheduler - INFO - Job ID: reminder_4, Hour: *
2025-05-22 15:39:54,905 - myfoodrepo.scheduler - INFO - Job ID: reminder_6, Hour: *
2025-05-22 15:39:54,908 - myfoodrepo.scheduler - INFO - Job ID: reminder_5, Hour: *
2025-05-22 15:39:54,909 - myfoodrepo.scheduler - INFO - Job ID: reminder_10, Hour: *
2025-05-22 15:39:54,912 - myfoodrepo.scheduler - INFO - Job ID: reminder_12, Hour: *
2025-05-22 15:39:54,914 - myfoodrepo.scheduler - INFO - Job ID: reminder_11, Hour: *
2025-05-22 15:39:54,916 - myfoodrepo.scheduler - INFO - Job ID: reminder_13, Hour: *
2025-05-22 15:39:54,918 - myfoodrepo.scheduler - INFO - Job ID: reminder_15, Hour: *
2025-05-22 15:39:54,921 - myfoodrepo.scheduler - INFO - Job ID: reminde

### Vanilla prompts

### Persona prompts

In [ ]:
os.makedirs("convo_logs", exist_ok=True)
def get_prompt(strategy):
    
    current_datetime = datetime.datetime.now() 
    base = CONVERSATION_ROLE.format(current_datetime=current_datetime) + USER_PROFILE.format(
        age=user_data["age"],
        gender=user_data["gender"],
        diet_goal=user_data["diet_goal"][5],
        diet_information=user_data["diet_info"][5],
        recent_meals=recent_meals,
        current_logging_streak=user_data["current_logging_streak"][5],
        prefered_language=user_data["prefered_language"]
    )
    
    if strategy == "vanilla":
        return base + CONVERSATION_RULES
    elif strategy == "vanilla_cot":
        return base + CONVERSATION_COT + CONVERSATION_RULES
    elif strategy == "vanilla_few_shots":
        return base + CONVERSATION_FEW_SHOTS + CONVERSATION_RULES
    elif strategy == "vanilla_few_shots_cot":
        return base + CONVERSATION_FEW_SHOTS + CONVERSATION_COT + CONVERSATION_RULES
    elif strategy == "vanilla_few_shots_cot":
        return base + CONVERSATION_FEW_SHOTS + CONVERSATION_COT + CONVERSATION_RULES
    elif strategy == "persona":
        return base + CONVERSATION_PERSONA + CONVERSATION_RULES
    elif strategy == "persona_cot":
        return base + CONVERSATION_PERSONA + CONVERSATION_COT + CONVERSATION_RULES
    elif strategy == "persona_few_shots":
        return base + CONVERSATION_PERSONA + CONVERSATION_FEW_SHOTS + CONVERSATION_RULES
    elif strategy == "persona_few_shots_panel":
        return base + CONVERSATION_PERSONA + CONVERSATION_FEW_SHOTS + PANEL_DISCUSSION + CONVERSATION_RULES
    elif strategy == "persona_few_shots_cot":
        return base + CONVERSATION_PERSONA + CONVERSATION_FEW_SHOTS + CONVERSATION_COT + CONVERSATION_RULES
    else:
        raise ValueError("Unknown prompt strategy")

In [8]:
from openai import OpenAI
def start_chat(strategy="vanilla", model=GPT_4_O, temperature=1, top_p=1):
    client = OpenAI()
    prompt = get_prompt(strategy)
    conversation_history = [{"role": "system", "content": f"[Prompt strategy: {strategy}]\n" + prompt}]

    
    print("=== Chat started. Type 'exit' to end. ===\n")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        
        conversation_history.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
            model=model,
            messages=conversation_history,
            temperature=temperature,
            top_p=top_p
        )

        assistant_message = response.choices[0].message.content.strip()
        print(f"\nAssistant: {assistant_message}\n")

        conversation_history.append({"role": "assistant", "content": assistant_message})
    
    return conversation_history

def save_conversation(conversation, strategy):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"convo_logs/convo_{strategy}_{timestamp}.txt"
    
    with open(filename, "w", encoding="utf-8") as f:
        for msg in conversation:
            role = msg['role'].upper()
            content = msg['content']
            f.write(f"{role}: {content}\n\n")
    
    print(f"Conversation saved to {filename}")

In [9]:
strategy = "vanilla_few_shots_cot" 

conversation = start_chat(strategy)
save_conversation(conversation, strategy)

=== Chat started. Type 'exit' to end. ===




Assistant: You started logging your meals on May 7th, 2025. Keep up the good work with your tracking!


Assistant: Your very first logged meal on May 7th, 2025, was a combination of short pastry base with butter, sweet, baked, spinach quiche with cake dough, and Skyr.

Conversation saved to convo_logs/convo_vanilla_few_shots_cot_20250522_154027.txt
